In [38]:
import sklearn as sk
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

print('numpy version:', np.__version__)
print('pandas version:', pd.__version__)
print('scikit-learn version:', sk.__version__)
print('matplotlib version:', matplotlib.__version__)

%matplotlib inline

numpy version: 1.12.1
pandas version: 0.20.1
scikit-learn version: 0.18.1
matplotlib version: 2.0.2


In [39]:
datafolder = 'data/'
filename = 'dataset_informa.csv'
df = pd.read_csv(datafolder + filename, sep=',', low_memory=False, encoding = 'ISO-8859-1')
print(df.shape)
df.head()

(190570, 49)


,AGENCY_ID,PRIMARY_AGENCY_ID,PROD_ABBR,PROD_LINE,STATE_ABBR,STAT_PROFILE_DATE_YEAR,RETENTION_POLY_QTY,POLY_INFORCE_QTY,PREV_POLY_INFORCE_QTY,NB_WRTN_PREM_AMT,...,PL_BOUND_CT_ELINKS,PL_QUO_CT_ELINKS,PL_BOUND_CT_PLRANK,PL_QUO_CT_PLRANK,PL_BOUND_CT_eQTte,PL_QUO_CT_eQTte,PL_BOUND_CT_APPLIED,PL_QUO_CT_APPLIED,PL_BOUND_CT_TRANSACTNOW,PL_QUO_CT_TRANSACTNOW
0,3,3,BOILERMACH,CL,IN,2005,0,0,0,40.0,...,0,0,0,103,50,288,0,0,0,0
1,3,3,BOILERMACH,CL,IN,2006,0,0,0,151.0,...,0,0,0,103,50,288,0,0,0,0
2,3,3,BOILERMACH,CL,IN,2007,0,0,0,40.0,...,0,0,0,103,50,288,0,0,0,0
3,3,3,BOILERMACH,CL,IN,2008,0,0,0,69.0,...,0,0,0,103,50,288,0,0,0,0
4,3,3,BOILERMACH,CL,IN,2009,0,0,0,28.0,...,0,0,0,103,50,288,0,0,0,0


In [40]:
df.drop(['RETENTION_RATIO'], axis = 1, inplace = True)

# Primary_Agency_ID: When this column data is missing/99999 we are going to replace it with the Agency_ID. 
# We can not find any way to associate a primary agency with the agency in question so will replace it with its ID.
df['PRIMARY_AGENCY_ID'] = np.where(df['PRIMARY_AGENCY_ID'] == 0, df['PRIMARY_AGENCY_ID'], df['AGENCY_ID'])

# (VENDOR)_END _YEAR: The year the agency stopped using the vendor. 
# This can either mean the vendor is never used or the vendor is still in use. 
# If the vendor has never been used we can replace the value with 0, if the vendor is still in use we can replace it with 1.
df['CL_END_YEAR'] = np.where(df['CL_START_YEAR'] == 0, 0, 1)
df['PL_END_YEAR'] = np.where(df['PL_START_YEAR'] == 0, 0, 1)

# Agency_appointment_year: 5000 missing data, irrelevant column, will be replaced with average.
mean = df['AGENCY_APPOINTMENT_YEAR'].mean()
df['AGENCY_APPOINTMENT_YEAR'].replace(99999, mean, inplace=True) 

# replace the rest of the 99999 values with 0
df.replace(99999, 0, inplace=True)


In [41]:
#Get a list of all values needing to be recalculated per item
recal = ["POLY_INFORCE_QTY", "NB_WRTN_PREM_AMT", "WRTN_PREM_AMT", "PRD_ERND_PREM_AMT", "PRD_INCRD_LOSSES_AMT"]
df = df[(df["MONTHS"] != 12)]
for index, row in df.iterrows():
    #Amount of months
    months = row["MONTHS"]
    for column in recal:
        newvalue = row[column] * (12 - months) #Value times amount of months missing
        df.set_value(index, column, newvalue) #Set new row in dataset
    row["MONTHS"] = 12 #Reset the months to be 12
df[recal].head(5)





,POLY_INFORCE_QTY,NB_WRTN_PREM_AMT,WRTN_PREM_AMT,PRD_ERND_PREM_AMT,PRD_INCRD_LOSSES_AMT
0,0,160.0,12298.52,12023.08,0.00
10,0,0.0,13459.81,10741.08,0.00
11,0,0.0,1446.72,864.80,0.00
21,0,0.0,0.00,3711.89,0.00
22,0,0.0,54429.88,70669.68,66670.04


In [42]:
df_old = df_filtered #Copy of the dataset for testing
df_incomplete[recal].head(5) #Get incomplete rows in the dataframe

,POLY_INFORCE_QTY,NB_WRTN_PREM_AMT,WRTN_PREM_AMT,PRD_ERND_PREM_AMT,PRD_INCRD_LOSSES_AMT
0,0,160.0,12298.52,12023.08,0.00
10,0,0.0,13459.81,10741.08,0.00
11,0,0.0,1446.72,864.80,0.00
21,0,0.0,0.00,3711.89,0.00
22,0,0.0,54429.88,70669.68,66670.04


In [43]:
#RETENTION_RATIO = RETENTION_POLY_QTY / PREV_POLY_INFORCE_QTYDataFrame.drop
#LOSS_RATIO = PRD_INCRD_LOSSES_AMT / WRTN_PREM_AMT

for index, row in df.iterrows():
    if (row["WRTN_PREM_AMT"] > 0) & (row["PRD_INCRD_LOSSES_AMT"] > 0):
        df.set_value(index, "LOSS_RATIO", row["PRD_INCRD_LOSSES_AMT"] / row["WRTN_PREM_AMT"])
    else:
        df.set_value(index, "LOSS_RATIO", 0)
    
df['LOSS_RATIO'] = df['LOSS_RATIO'].replace(99998, 0)
df['LOSS_RATIO'] = df['LOSS_RATIO'].replace(99997, 0)


In [44]:
df["LOSS_RATIO"]

0         0.000000
10        0.000000
11        0.000000
21        0.000000
22        1.224879
32        0.030583
33        0.000000
43        0.000000
44        0.717888
54        0.273193
55        0.046221
65        0.000000
66        0.000000
68        0.000000
71        0.000000
72        0.000000
75        0.000000
76        0.000000
82        0.000000
83        0.000000
90        0.000000
91        0.000000
93        0.000000
94        0.000000
98        0.000000
99        0.249880
109       0.000000
110       0.000000
120       0.000000
121       0.000000
            ...   
190429    0.000000
190432    0.000000
190433    0.000000
190443    0.543867
190444    0.000000
190454    0.000000
190455    0.000000
190465    0.000000
190466    0.000000
190476    0.000000
190477    0.000000
190487    0.000000
190488    0.000000
190498    0.000000
190499    0.000000
190509    0.000000
190510    0.000000
190520    0.000000
190521    0.000000
190524    0.000000
190525    0.000000
190535    0.

In [45]:
df[["PRD_INCRD_LOSSES_AMT","WRTN_PREM_AMT","LOSS_RATIO"]]

,PRD_INCRD_LOSSES_AMT,WRTN_PREM_AMT,LOSS_RATIO
0,0.00,12298.52,0.000000
10,0.00,13459.81,0.000000
11,0.00,1446.72,0.000000
21,0.00,0.00,0.000000
22,66670.04,54429.88,1.224879
32,3445.40,112658.00,0.030583
33,0.00,7187.64,0.000000
43,0.00,2627.94,0.000000
44,292650.28,407654.68,0.717888
54,125561.73,459608.17,0.273193


In [46]:
df[["RETENTION_POLY_QTY","PREV_POLY_INFORCE_QTY"]]

,RETENTION_POLY_QTY,PREV_POLY_INFORCE_QTY
0,0,0
10,0,0
11,0,0
21,0,0
22,0,0
32,0,0
33,0,0
43,0,0
44,0,0
54,0,0


In [47]:
df.head(50)

,AGENCY_ID,PRIMARY_AGENCY_ID,PROD_ABBR,PROD_LINE,STATE_ABBR,STAT_PROFILE_DATE_YEAR,RETENTION_POLY_QTY,POLY_INFORCE_QTY,PREV_POLY_INFORCE_QTY,NB_WRTN_PREM_AMT,...,PL_BOUND_CT_ELINKS,PL_QUO_CT_ELINKS,PL_BOUND_CT_PLRANK,PL_QUO_CT_PLRANK,PL_BOUND_CT_eQTte,PL_QUO_CT_eQTte,PL_BOUND_CT_APPLIED,PL_QUO_CT_APPLIED,PL_BOUND_CT_TRANSACTNOW,PL_QUO_CT_TRANSACTNOW
0,3,3,BOILERMACH,CL,IN,2005,0,0,0,160.0,...,0,0,0,103,50,288,0,0,0,0
10,3,3,BOILERMACH,CL,IN,2015,0,0,0,0.0,...,0,0,0,103,50,288,0,0,0,0
11,3,3,BOILERMACH,CL,KY,2005,0,0,0,0.0,...,0,0,0,103,50,288,0,0,0,0
21,3,3,BOILERMACH,CL,KY,2015,0,0,0,0.0,...,0,0,0,103,50,288,0,0,0,0
22,3,3,BOP,CL,IN,2005,0,0,0,0.0,...,0,0,0,103,50,288,0,0,0,0
32,3,3,BOP,CL,IN,2015,0,0,0,0.0,...,0,0,0,103,50,288,0,0,0,0
33,3,3,BOP,CL,KY,2005,0,0,0,0.0,...,0,0,0,103,50,288,0,0,0,0
43,3,3,BOP,CL,KY,2015,0,0,0,0.0,...,0,0,0,103,50,288,0,0,0,0
44,3,3,COMMAUTO,CL,IN,2005,0,0,0,196.0,...,0,0,0,103,50,288,0,0,0,0
54,3,3,COMMAUTO,CL,IN,2015,0,0,0,0.0,...,0,0,0,103,50,288,0,0,0,0


In [48]:
average = ["PREV_POLY_INFORCE_QTY","POLY_INFORCE_QTY", "NB_WRTN_PREM_AMT", "WRTN_PREM_AMT", "PRD_ERND_PREM_AMT"]

states = df.STATE_ABBR.unique()
prod_line = df.PROD_LINE.unique()
   
state_dict = {}    
for state in states:
    for line in prod_line:
        state_mean = df[(df["STATE_ABBR"] == state) & (df["PROD_LINE"] == line)].mean()
        state_dict[(state, line)] = state_mean


In [49]:
for index, row in df.iterrows():
    for col in average:
        if(row[col] == 0):
            try:
                newvalue = state_dict[(row["STATE_ABBR"], row["PROD_LINE"])]
            except(ValueError): #No average can be found since its all 0
                newvalue = 0 
            df.set_value(index, col, newvalue[col])

In [50]:
df[average].head(10)

,PREV_POLY_INFORCE_QTY,POLY_INFORCE_QTY,NB_WRTN_PREM_AMT,WRTN_PREM_AMT,PRD_ERND_PREM_AMT
0,5,28,160.000000,12298.520000,12023.08
10,5,28,6684.717013,13459.810000,10741.08
11,8,47,9354.245527,1446.720000,864.80
21,8,47,9354.245527,35563.967031,3711.89
22,5,28,6684.717013,54429.880000,70669.68
32,5,28,6684.717013,112658.000000,65653.77
33,8,47,9354.245527,7187.640000,10013.52
43,8,47,9354.245527,2627.940000,7097.23
44,5,28,196.000000,407654.680000,1150966.64
54,5,28,6684.717013,459608.170000,333340.56


In [51]:
df.replace(99999, 0, inplace=True)
df.STATE_ABBR.unique()
statedict = {np.NaN : 0, "IN" : 1, "KY" : 2, "MI" : 3, "OH" : 4, "PA" : 5, "WV" : 6}
df.PROD_ABBR.unique()
proddict = {np.NaN : 0, "BOILERMACH" : 1, "BOP" : 2, "COMMAUTO" : 3, "COMMINLMAR" : 4, "COMMPOL": 5, "COMMUMBREL" : 6, "CRIME" : 7,
           "FIREALLIED" : 8, "GARAGE" : 9, "GENERALIAB" : 10, "WORKCOMP" : 11, "ANNIV" : 12, "ANNIV   12":13, "CYCLES":14,
           "DTALK":15, "DWELLFIRE":16, "HOMEONWERS":17,"MOBILEHOME":18,"MOTORHOM12":19,"MOTORHOME":20,"PERSUMBREL":21, "YACHT":22,
           "DTALK   12":23, "PERSINLMAR":24, "SNOWMOBILE":25,"CYCLES  12":26,"SNOWMOBBI12":27,"PERSAIP":28}
df.PROD_LINE.unique()
prodlinedict = {np.NaN : 0, "CL":1, "PL":2}
vendordict = {'Unknown': 0, 'A': 1, 'B':2, 'C' : 3, 'E':4}

df['STATE_ID'] = df['STATE_ABBR'].map(statedict)
df['PROD_ID'] = df["PROD_ABBR"].map(proddict)
df['PROD_LINE_ID'] = df['PROD_LINE'].map(prodlinedict)
df['VENDOR_ID'] = df['VENDOR'].map(vendordict)
df[["STATE_ABBR", "PROD_ABBR", "PROD_LINE", "VENDOR", "STATE_ID", "PROD_ID", "PROD_LINE_ID", "VENDOR_ID"]].head(50)

,STATE_ABBR,PROD_ABBR,PROD_LINE,VENDOR,STATE_ID,PROD_ID,PROD_LINE_ID,VENDOR_ID
0,IN,BOILERMACH,CL,Unknown,1,1.0,1,0.0
10,IN,BOILERMACH,CL,Unknown,1,1.0,1,0.0
11,KY,BOILERMACH,CL,Unknown,2,1.0,1,0.0
21,KY,BOILERMACH,CL,Unknown,2,1.0,1,0.0
22,IN,BOP,CL,Unknown,1,2.0,1,0.0
32,IN,BOP,CL,Unknown,1,2.0,1,0.0
33,KY,BOP,CL,Unknown,2,2.0,1,0.0
43,KY,BOP,CL,Unknown,2,2.0,1,0.0
44,IN,COMMAUTO,CL,Unknown,1,3.0,1,0.0
54,IN,COMMAUTO,CL,Unknown,1,3.0,1,0.0


In [52]:
def ProgressBar(count, total):
    complete = ["=", "=", "=", "=","=","=","=","=","=","="]
    empty = ["_","_","_","_","_","_","_","_","_","_"]
    percentage = int(count/total*10)
    if(percentage%1 == 0):
        clear_output()
        output = "|";
        for index in range(0, percentage):
            output += complete[index]
        for index in range(percentage, 10):
            output += empty[index]
        output += "|"
        print(output)
        print(str(percentage*10) + "%")

In [53]:
#POLY_GROWTH_RATIO = 
#PREM_GROWTH_RATIO = WRTN_PREM_AMT (Current)- WRTN_PREM_AMT (Last Year) / (Current + Last year)

#index, row, df.itter
#try:
#   prevyear = df where(agencyid == id, prodid == prodid, prodline == prodline, year == currentyear-1)[columnname]
#except:
#prevyear = 0
#if prevyear > 0
#df put growthratio (this-prev/prev)

column = "POLY_INFORCE_QTY"
columnname= "POLY_GROWTH_RATIO"
def GrowthRatioCalculator(df, column, columnname):
    for index, row in df.iterrows():
        if(index > 0): #Start off at 1 so we don't get an index of -1
            prevyear = df.loc[index-1] #Get the previous item in the dataframe
            if(prevyear["STATE_ABBR"] == row["STATE_ABBR"]) & (prevyear["PROD_LINE"] == row["PROD_LINE"]) & (prevyear["STAT_PROFILE_DATE_YEAR"] == (row["STAT_PROFILE_DATE_YEAR"]-1)) & (prevyear["AGENCY_ID"] == row["AGENCY_ID"])& (prevyear["PROD_ABBR"] == row["PROD_ABBR"]):
                prevyear = prevyear[column] #Get the actual value of the previous year
                try:
                    if(prevyear > 0): #Check if its bigger than 0/ works
                        df.set_value(index, columnname, (row[column] -prevyear)/prevyear) #Set the value in the column to be the percentage
                        print((row[column] -prevyear)/prevyear) #Print out the value so we know it's working

                except:
                    #Can go wrong when we continue to a new product.
                    #If there is a missmatch within the if statement and it returns false, it will generate a kind of null value.
                    #We can not work with this value so it will throw an exception.
                    #If it goes wrong, put a 0 in the dataframe.
                    df.set_value(index, columnname, 0)
    

In [54]:
df.head(50)

,AGENCY_ID,PRIMARY_AGENCY_ID,PROD_ABBR,PROD_LINE,STATE_ABBR,STAT_PROFILE_DATE_YEAR,RETENTION_POLY_QTY,POLY_INFORCE_QTY,PREV_POLY_INFORCE_QTY,NB_WRTN_PREM_AMT,...,PL_BOUND_CT_eQTte,PL_QUO_CT_eQTte,PL_BOUND_CT_APPLIED,PL_QUO_CT_APPLIED,PL_BOUND_CT_TRANSACTNOW,PL_QUO_CT_TRANSACTNOW,STATE_ID,PROD_ID,PROD_LINE_ID,VENDOR_ID
0,3,3,BOILERMACH,CL,IN,2005,0,28,5,160.000000,...,50,288,0,0,0,0,1,1.0,1,0.0
10,3,3,BOILERMACH,CL,IN,2015,0,28,5,6684.717013,...,50,288,0,0,0,0,1,1.0,1,0.0
11,3,3,BOILERMACH,CL,KY,2005,0,47,8,9354.245527,...,50,288,0,0,0,0,2,1.0,1,0.0
21,3,3,BOILERMACH,CL,KY,2015,0,47,8,9354.245527,...,50,288,0,0,0,0,2,1.0,1,0.0
22,3,3,BOP,CL,IN,2005,0,28,5,6684.717013,...,50,288,0,0,0,0,1,2.0,1,0.0
32,3,3,BOP,CL,IN,2015,0,28,5,6684.717013,...,50,288,0,0,0,0,1,2.0,1,0.0
33,3,3,BOP,CL,KY,2005,0,47,8,9354.245527,...,50,288,0,0,0,0,2,2.0,1,0.0
43,3,3,BOP,CL,KY,2015,0,47,8,9354.245527,...,50,288,0,0,0,0,2,2.0,1,0.0
44,3,3,COMMAUTO,CL,IN,2005,0,28,5,196.000000,...,50,288,0,0,0,0,1,3.0,1,0.0
54,3,3,COMMAUTO,CL,IN,2015,0,28,5,6684.717013,...,50,288,0,0,0,0,1,3.0,1,0.0


In [55]:
# split PL and CL datasets
df_cl = df[df.PROD_LINE=='CL']
df_pl = df[df.PROD_LINE=='PL']

# remove irrelevant columns from splits
df_cl = df_cl.filter(regex='^(?!PL_)\w+', axis=1)
df_pl = df_pl.filter(regex='^(?!CL_)\w+', axis=1)

# write dataframes to csv files
df_cl.to_csv('cleaned_CL_'+filename)
df_pl.to_csv('cleaned_PL_'+filename)